In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras-tuner
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import keras
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler

In [4]:
ipl = pd.read_csv('/content/drive/MyDrive/Projects/ipl_data.csv')
ipl.head()

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
0,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,1,0,0.1,1,0,0,0,222
1,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,1,0,0.2,1,0,0,0,222
2,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.2,2,0,0,0,222
3,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.3,2,0,0,0,222
4,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.4,2,0,0,0,222


In [24]:
# selecting features and target variable
features = ['venue', 'bat_team', 'bowl_team', 'batsman', 'bowler', 'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5']
target = 'total'

# Extract data
X = ipl[features]
y = ipl[target]

# Encode Categorical Variables
categorical_features = ['venue', 'bat_team', 'bowl_team', 'batsman', 'bowler']
encoder = OneHotEncoder(sparse_output=False, drop='first',handle_unknown='ignore')
X_encoded = encoder.fit_transform(X[categorical_features])

# Concatenate encoded categorical features with numerical features
numerical_features = ['runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5']
X_numerical = X[numerical_features].values
X_preprocessed = np.hstack([X_encoded, X_numerical])

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# scale numerical data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Display shapes of all prepared data
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((60811, 803), (15203, 803), (60811,), (15203,))

In [25]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [7]:
model = Sequential([
    LSTM(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)  # Final output layer
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
# Define the neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1) # output layer for regression
])

# compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# Display the model summary
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 128)                 │         102,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 113,281 (442.50 KB)

 Trainable params: 113,281 (442.50 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# training the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    verbose=1)

Epoch 1/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 3673.4294 - mae: 40.0350 - val_loss: 343.6679 - val_mae: 14.0701
Epoch 2/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - loss: 534.4210 - mae: 18.0054 - val_loss: 264.0155 - val_mae: 12.2256
Epoch 3/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 455.0174 - mae: 16.6322 - val_loss: 226.8102 - val_mae: 11.3314
Epoch 4/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 399.8028 - mae: 15.6077 - val_loss: 221.5179 - val_mae: 11.2875
Epoch 5/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - loss: 361.1886 - mae: 14.8297 - val_loss: 187.8321 - val_mae: 10.1402
Epoch 6/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 334.5474 - mae: 14.2432 - val_loss: 182.0692 - val_mae: 10.0576
Epoch 7/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 311.5869 - mae: 13.6752 - val_loss: 170.7005 - val_mae: 9.7551
Epoch 8/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 292.6881 - mae: 13.1595 - val_loss: 150.3994 - v

In [13]:
# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss (MSE): {test_loss}")
print(f"Test Mean Absolute Error (MAE): {test_mae}")

Test Loss (MSE): 123.31510925292969
Test Mean Absolute Error (MAE): 8.52673625946045


In [26]:
# Reshaping data for LSTM (assuming sequential data like time series)
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))  # Reshaping for LSTM
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define the LSTM model
model = Sequential([
    LSTM(128, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile and train the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_data=(X_test_reshaped, y_test))

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss (MSE): {test_loss}')
print(f'Test MAE: {test_mae}')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 28s 13ms/step - loss: 5163.0083 - mae: 45.9542 - val_loss: 338.0495 - val_mae: 13.8458
Epoch 2/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 42s 14ms/step - loss: 297.3029 - mae: 12.8383 - val_loss: 212.1031 - val_mae: 10.7155
Epoch 3/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - loss: 191.0297 - mae: 10.1464 - val_loss: 151.0819 - val_mae: 8.9638
Epoch 4/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - loss: 136.8642 - mae: 8.5057 - val_loss: 117.6276 - val_mae: 7.8554
Epoch 5/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step - loss: 105.1164 - mae: 7.3466 - val_loss: 93.2546 - val_mae: 6.8708
Epoch 6/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - loss: 83.9384 - mae: 6.5189 - val_loss: 79.7670 - val_mae: 6.2996
Epoch 7/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 42s 12ms/step - loss: 69.9376 - mae: 5.8927 - val_loss: 73.6643 - val_mae: 6.0941
Epoch 8/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 26s 14ms/step - loss: 59.6875 - mae: 5.4285 - val_loss: 64.8749 - val_mae: 

In [19]:
# Define the model for tuning
def build_model(hp):
    model = Sequential()
    model.add(
        LSTM(
            units=hp.Int("units", min_value=32, max_value=256, step=32),
            activation='relu',
            input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])
        )
    )
    model.add(Dropout(hp.Float("dropout", min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(
        units=hp.Int("dense_units", min_value=16, max_value=128, step=16),
        activation='relu'
    ))
    model.add(Dense(1))  # Output layer for regression
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice("learning_rate", [0.001, 0.01, 0.1])),
        loss="mse",
        metrics=["mae"]
    )
    return model

# Initialize tuner
tuner = kt.Hyperband(
    build_model,
    objective="val_mae",
    max_epochs=20,
    factor=3,
    directory="lstm_tuning",
    project_name="ipl_score_predictor"
)

# Run the hyperparameter search
tuner.search(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test), epochs=20, batch_size=32)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Optimal Parameters:
LSTM Units: {best_hps.get('units')}
Dropout: {best_hps.get('dropout')}
Dense Units: {best_hps.get('dense_units')}
Learning Rate: {best_hps.get('learning_rate')}
""")

# Train the best model
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test), epochs=50, batch_size=32)

# Evaluate the model
test_loss, test_mae = best_model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss (MSE): {test_loss}')
print(f'Test MAE: {test_mae}')

Trial 30 Complete [00h 05m 31s]
val_mae: 18.368192672729492

Best val_mae So Far: 3.7629992961883545
Total elapsed time: 02h 00m 04s

Optimal Parameters:
LSTM Units: 96
Dropout: 0.2
Dense Units: 96
Learning Rate: 0.001

Epoch 1/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - loss: 6080.5054 - mae: 52.9680 - val_loss: 647.8403 - val_mae: 15.4857
Epoch 2/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - loss: 630.4338 - mae: 17.0380 - val_loss: 326.1821 - val_mae: 13.0776
Epoch 3/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 41s 11ms/step - loss: 279.9470 - mae: 12.0782 - val_loss: 164.5438 - val_mae: 9.2062
Epoch 4/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 40s 11ms/step - loss: 181.6693 - mae: 9.9243 - val_loss: 117.3284 - val_mae: 7.6931
Epoch 5/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - loss: 140.2669 - mae: 8.6248 - val_loss: 95.2643 - val_mae: 6.8484
Epoch 6/50
1901/1901 ━━━━━━━━━━━━━━━━━━━━ 39s 10ms/step - loss: 113.6470 - mae: 7.7740 - val_loss: 80.9476 - val_mae: 6.1853
Epoch 7/50
1901/1901

In [37]:
custom_input = {
    'venue': 'M. Chinnaswamy Stadium',
    'bat_team': 'Royal Challengers Bangalore',
    'bowl_team': 'Chennai Super Kings',
    'batsman': 'V Kohli',
    'bowler': 'DJ Bravo',
    'runs': 100,
    'wickets': 2,
    'overs': 12.4,
    'runs_last_5': 45,
    'wickets_last_5': 1
}


# Convert to DataFrame
custom_input_df = pd.DataFrame([custom_input])

# Encode categorical features using the same encoder as training
custom_encoded = encoder.transform(custom_input_df[categorical_features])

# Extract and scale numerical features
custom_numerical = custom_input_df[numerical_features].values

# Combine encoded categorical features and numerical features
custom_combined = np.hstack([custom_encoded, custom_numerical])

# Scale the combined features using the same scaler as training
custom_scaled = scaler.transform(custom_combined)

# Reshape to match LSTM input shape (samples, timesteps, features)
custom_preprocessed = custom_scaled.reshape((1, 1, custom_scaled.shape[1]))

# Predict the score
predicted_total = model.predict(custom_preprocessed)
rounded_score = round(predicted_total[0][0])
#print(f"Predicted Total Score: {predicted_total[0][0]:.2f}")
print(f"Predicted Total Score: {rounded_score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted Total Score: 145


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [38]:
# saving the model
model.save('ipl_score_predictor.h5')

In [39]:
# saving preprocessing objects
import joblib

joblib.dump(encoder, 'encoder.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [40]:
#downloading the files
from google.colab import files

files.download('ipl_score_predictor.h5')
files.download('encoder.pkl')
files.download('scaler.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>